<a href="https://colab.research.google.com/github/rprimi/colB5BERT/blob/main/python/colB5BERT_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **colB5BERT:** Fine tuning colBERT with Big Five dataset

* Notebook inspirado no buscador denso de Leandro Carísio Fernandes

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip3 install transformers
!pip install rpy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!git clone https://github.com/rprimi/colB5BERT.git

%cd /content/colB5BERT
!git pull


Cloning into 'colB5BERT'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 242 (delta 34), reused 18 (delta 18), pack-reused 196
Receiving objects: 100% (242/242), 32.67 MiB | 4.33 MiB/s, done.
Resolving deltas: 100% (149/149), done.
/content/colB5BERT
Already up to date.


### Libraries

In [12]:
import sys
sys.path.append('/content/colB5BERT/python/')

import utils
import vsm
import sst

import os
import pandas as pd
import numpy as np
import textwrap
import pickle
import h5py
import logging

from transformers import BertModel, BertTokenizer
from transformers import RobertaModel, RobertaTokenizer

import torch
from torch.nn.functional import cosine_similarity
from tqdm import tqdm

import pandas as pd

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 26 17:30:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
df_test = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_test.tsv', sep='\t')
df_train = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_train.tsv', sep='\t')


# Separa os conjuntos de treinamento e validação
queries_train = df_train['item_pt_text'].tolist()
docs_train = df_train['texto_dividido'].tolist()
positive_ex_train = df_train['postive_ex'].tolist()


queries_val = df_test['item_pt_text'].tolist()
docs_val = df_test['texto_dividido'].tolist()
positive_ex_val = df_test['postive_ex'].tolist()


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Preparação do ambiente

### Variáveis para controlar o fluxo do caderno




In [ ]:
treinar_e_salvar_modelos = False

gerar_e_salvar_matriz_docs_trec_covid = False

# Local onde fica o arquivo que contém a matriz de todos os documentos do trec_covid. Se gerar_e_salvar_matriz_docs_trec_covid = True, esse arquivo será sobrescrito com a nova geração
arquivo_matriz_docs_trec_covid = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula6-buscador-denso/matriz_docs_trec_covid.pt'

# Diretório onde vai salvando o modelo a cada época
dir_modelos = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula6-buscador-denso/modelos/'

# Nome dos modelos e tokenizador. São esses modelos que serão carregados no início com o from_pretrained.
# Se quiser iniciar um treinamento do 0, é necessário substituir por "microsoft/MiniLM-L12-H384-uncased"
nome_modelo_query = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula6-buscador-denso/modelos/final/query/' #nome_modelo_query = "microsoft/MiniLM-L12-H384-uncased"
nome_modelo_doc = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula6-buscador-denso/modelos/final/doc/' #nome_modelo_doc = "microsoft/MiniLM-L12-H384-uncased"
nome_tokenizador = "microsoft/MiniLM-L12-H384-uncased"

# Gerar e salvar arquivo de embeddings para os documentos usando os modelos da biblioteca Sequence Transformers:
gerar_arquivo_doc_embeddings_all_mpnet_base_v2 = False
arq_doc_embeddings_all_mpnet_base_v2 = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula6-buscador-denso/docs_embeddings-all-mpnet-base-v2.pt'

gerar_arquivo_doc_embeddings_all_MiniLM_L12_v2 = False
arq_doc_embeddings_all_MiniLM_L12_v2 = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula6-buscador-denso/docs_embeddings-all-MiniLM-L12-v2.pt'


url_ms_marco_treinamento = "https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv"
url_trec_covid = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

max_length = 256
batch_size = 32
epochs = 20
lr = 2e-5

### Instalação de libs e montagem do Drive

In [ ]:
# Já monta o drive, pois vamos usar o índice invertido da Aula 1 para usar o BM25 implementado também na aula 1
# Além disso, é necessário para salvar/recuperar o modelo tunado
from google.colab import drive
drive.mount('/content/drive')

!pip install transformers datasets -q
!pip install sentence-transformers -q
!pip install pyserini -q
!pip install faiss-gpu -q

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Fine-tuning dos encoders

Cada linha do dataset de treino (MSMARCO-tiny) possui 3 campos: query, exemplo positivo, exemplo negativo. Vamos desconsiderar os exemplos negativos e usar apenas os positivos. Para uma dada query, usamos os exemplos positivos de outras queries como negativo para a query avaliada.

In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

# Só faz o download se ainda não tiver feito
if not Path('./collections/msmarco_triples.train.tiny.tsv').is_file():
  !wget {url_ms_marco_treinamento} -P collections # type: ignore

# Lê usando pandas
msmarco_df = pd.read_csv("collections/msmarco_triples.train.tiny.tsv", sep='\t', names=['query', 'relevante', 'nao_relevante'], header=None)
msmarco_train_df, msmarco_val_df = train_test_split(msmarco_df, test_size=0.1, random_state=42)

# Separa os conjuntos de treinamento e validação
queries_train = msmarco_train_df['query'].tolist()
docs_train = msmarco_train_df['relevante'].tolist()
queries_val = msmarco_val_df['query'].tolist()
docs_val = msmarco_val_df['relevante'].tolist()

print('Total treinamento', len(queries_train), len(docs_train))
print('Total validação', len(queries_val), len(docs_val))

Total treinamento 9900 9900
Total validação 1100 1100


Define os datasets e dataloaders:

In [19]:
from transformers import AutoTokenizer
from torch.utils import data
from torch.utils.data import DataLoader
from transformers import BatchEncoding

In [31]:
max_length = 512
batch_size = 32
epochs = 20
lr = 2e-5

In [32]:
# Definição do Dataset
class Dataset(data.Dataset):
    # Recebe dois vetores de textos e um vetor de labels
    def __init__(self, tokenizer, textos, labels, max_seq_length = max_length):
        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer
        self.textos = textos
        self.labels = labels
        self.cache = {}

    def __len__(self):
        return len(self.textos)

    def __getitem__(self, idx):
        self.cache[str(idx)] = self.cache.get(str(idx),
                   (self.tokenizer(self.textos[idx],
                                  padding=True,
                                  truncation=True,
                                  max_length=self.max_seq_length
                                  ),
                    self.labels[idx])
                   )
        return self.cache[str(idx)]





In [28]:
bert_weights_name = 'neuralmind/bert-base-portuguese-cased'
#bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
#bert_model = BertModel.from_pretrained(bert_weights_name)



tokenizer = AutoTokenizer.from_pretrained(bert_weights_name)

# Temos 2 datasets de cada tipo (train/val). Um pro encoder do documentos e outro pro encoder das queries
# Datasets de treinamento
dataset_queries_train = Dataset(tokenizer, queries_train)
dataset_docs_train = Dataset(tokenizer, docs_train)

# Datasets de validação
dataset_queries_val = Dataset(tokenizer, queries_val)
dataset_docs_val = Dataset(tokenizer, docs_val)

Sure, in PyTorch, `Dataset` and `DataLoader` are essential components for working with batched data. Let's dive into them.

**1. Dataset**

In PyTorch, a dataset is represented by a subclass of the `torch.utils.data.Dataset` class. The dataset class is a way to provide an interface for accessing all the training or testing samples in your dataset. You need to override two methods when subclassing `Dataset`:

* `__len__` so that len(dataset) returns the size of the dataset, and
* `__getitem__` to support the indexing such that dataset[i] can be used to get the i-th sample.

For example, let's say you have a list of images and their respective labels. Your dataset class would look something like this:

```python
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = read_image(self.image_paths[idx])  # Some function to read image from path
        label = self.labels[idx]
        return image, label
```

**2. DataLoader**

While the `Dataset` class is used to provide an interface for accessing the data, the `DataLoader` is used to load the data in batches during training. This is beneficial because it can significantly reduce the amount of memory used (especially for large datasets).

The `DataLoader` takes in a dataset and a variety of optional parameters. One of the most important ones is `batch_size`, which determines how many samples per batch to load. Other parameters like `shuffle` determine whether data is served in a random order. Here's an example of using a `DataLoader` with the previous `MyDataset` class:

```python
from torch.utils.data import DataLoader

dataset = MyDataset(image_paths, labels)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

for batch in dataloader:
    images, labels = batch
    # Do something with these batches of images and labels...
```

In this code, `dataloader` is an iterator — we can loop over it and fetch batches of images and labels.

Additionally, `DataLoader` is typically used in conjunction with the `Dataset` class. The `Dataset` retrieves our dataset's items, while the `DataLoader` wraps an iterable around the `Dataset` to enable easy access to the samples.

**3. collate_fn**

The `collate_fn` is a function that the `DataLoader` applies to a list of samples from the `Dataset` to form a batch. The `collate_fn` is necessary because the number of samples in different items in the dataset may not be the same. So, we need a function to correctly collate these samples into a batch.

The default `collate_fn` should work fine for most use cases. However, if you have a complex structure of the data, or the data is not a tensor, you might need to write your own `collate_fn`.

**Note:** The examples provided in this explanation are relatively basic. Depending on the complexity of your dataset and what you want to do with your data, your `Dataset` and `DataLoader` implementations could become more complex. For example, you may need to transform your data (like resizing images, or tokenizing text), in which case these transformations would typically be implemented in your `Dataset` class.


In [44]:
# Create datasets
dataset_queries_train = Dataset(tokenizer, queries_train, positive_ex_train)
dataset_docs_train = Dataset(tokenizer, docs_train, positive_ex_train)

dataset_queries_val = Dataset(tokenizer, queries_val, positive_ex_val)
dataset_docs_val = Dataset(tokenizer, docs_val, positive_ex_val)


# Dataloaders para os datasets

#collate_fn = lambda batch: BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))
def collate_fn(batch):
    #print('Dentro de collate_fn')
    #print(BatchEncoding(tokenizer.pad(batch, return_tensors='pt')))
    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

# collate function that also handles the labels
def collate_fn(batch):
    inputs = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    return BatchEncoding(tokenizer.pad(inputs, return_tensors='pt')), torch.tensor(labels)


# Create dataloaders
dataloader_queries_train = DataLoader(dataset_queries_train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_docs_train = DataLoader(dataset_docs_train, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

dataloader_queries_val = DataLoader(dataset_queries_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
dataloader_docs_val = DataLoader(dataset_docs_val, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)



# Adapt your DataLoader object
class PositiveExDataLoader:
    def __init__(self, dataloader, positive_ex):
        self.dataloader = dataloader
        self.positive_ex = positive_ex

    def __iter__(self):
        for batch, pos_ex in zip(self.dataloader, self.positive_ex):
            yield batch, pos_ex
    def __len__(self):
        return len(self.dataloader)
# Wrap your original dataloaders
dataloader_queries_train = PositiveExDataLoader(dataloader_queries_train, positive_ex_train)
dataloader_docs_train = PositiveExDataLoader(dataloader_docs_train, positive_ex_train)

dataloader_queries_val = PositiveExDataLoader(dataloader_queries_val, positive_ex_val)
dataloader_docs_val = PositiveExDataLoader(dataloader_docs_val, positive_ex_val)


Carrega os modelos:

In [36]:
# Agora vamos carregar dois modelos:
import torch
from transformers import AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Se tiver que treinar os modelos, abre
model_query = AutoModel.from_pretrained(bert_weights_name).to(device)
model_doc = AutoModel.from_pretrained(bert_weights_name).to(device)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predicti

Define função pro cálculo da loss (modifiquei essa função para trabalhar com um vetor indicando quais pares são relevantes e quais não são:

In [45]:
# Essa função já considera o resultado via batchs:
def compute_loss_com_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs, positive_ex):
    outputs_queries = model_query(**tokenized_queries.to(device))
    outputs_docs    = model_doc(**tokenized_docs.to(device))

    # Extrai a última camada oculta associada ao token [CLS]
    tcls_queries = outputs_queries.last_hidden_state[:, 0, :]
    tcls_docs    = outputs_docs.last_hidden_state[:, 0, :]

    # Normaliza os tensores
    #tcls_queries = tcls_queries / torch.norm(tcls_queries, dim=1, keepdim=True)
    #tcls_docs = tcls_docs / torch.norm(tcls_docs, dim=1, keepdim=True)

    # Agora é necessário calcular a loss. Para isso, o primeiro passo é
    # calcular a similaridade entre uma query e documento (sim(q, d))
    similaridade = torch.matmul(tcls_queries, torch.transpose(tcls_docs, 0, 1))

    # Calcula a exponencial da similaridade
    exp_sim = torch.exp(similaridade)

    # Calcula a loss
    # We are now considering only the positive examples (where positive_ex is 1)
    positive_exp_sim = exp_sim * positive_ex.to(device)
    soma_linhas = positive_exp_sim.sum(dim=1)
    diagonal = torch.diag(positive_exp_sim)
    log_loss = -1* torch.log(diagonal/soma_linhas)

    loss = torch.mean(log_loss)
    return loss

def compute_loss_sem_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs):
    with torch.no_grad():
        return compute_loss_com_gradiente(model_query, tokenized_queries, model_doc, tokenized_docs)

def compute_loss_dataloaders(model_query, dataloader_query, model_doc, dataloader_docs):
    loss = 0
    n_batches = 0
    for batch_query, batch_docs in zip(dataloader_query, dataloader_docs):
        loss = loss + compute_loss_sem_gradiente(model_query, batch_query, model_doc, batch_docs)
        n_batches += 1
    return loss/n_batches

In [ ]:
%%time
# Só pra medir o tempo que ele demora para calcular a loss em todo o dataset de treinamento
model_query.eval()
model_doc.eval()
print(f'Loss de treinamento: {compute_loss_dataloaders(model_query, dataloader_queries_train, model_doc, dataloader_docs_train)}')
print(f'Loss de validação: {compute_loss_dataloaders(model_query, dataloader_queries_val, model_doc, dataloader_docs_val)}')

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Loss de treinamento: 0.0008150112116709352
Loss de validação: 0.04865604639053345
CPU times: user 41.7 s, sys: 119 ms, total: 41.8 s
Wall time: 42.1 s


Agora treina os dois encoders simulatenamente:

In [47]:
%%time
from transformers import Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, AdamW
from tqdm.auto import tqdm

treinar_e_salvar_modelos  = True


if treinar_e_salvar_modelos:
  # Training loop
  optimizer_query = AdamW(model_query.parameters(), lr=lr)
  optimizer_doc = AdamW(model_doc.parameters(), lr=lr)

  num_training_steps = epochs * len(dataloader_queries_train)
  num_warmup_steps = int(num_training_steps * 0.1)

  # get_linear_schedule_with_warmup get_cosine_with_hard_restarts_schedule_with_warmup
  scheduler_query = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer_query, num_warmup_steps, num_training_steps)
  scheduler_doc = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer_doc, num_warmup_steps, num_training_steps)

  for epoch in tqdm(range(epochs), desc='Epochs'):
      model_query.train()
      model_doc.train()

      train_losses = []
      for (batch_query, positive_ex_query), (batch_docs, positive_ex_docs) in tqdm(list(zip(dataloader_queries_train, dataloader_docs_train)), mininterval=0.5, desc='Train', disable=False):
        optimizer_query.zero_grad()
        optimizer_doc.zero_grad()

        # Ensure positive_ex_query and positive_ex_docs are tensors and on the same device as your models
        positive_ex_query = torch.tensor(positive_ex_query).to(device)
        positive_ex_docs = torch.tensor(positive_ex_docs).to(device)

        loss = compute_loss_com_gradiente(model_query, batch_query, model_doc, batch_docs, positive_ex_query, positive_ex_docs)
        loss.backward()

        optimizer_query.step()
        optimizer_doc.step()

        scheduler_query.step()
        scheduler_doc.step()


      model_query.save_pretrained(f'{dir_modelos}{epoch}/query/')
      model_doc.save_pretrained(f'{dir_modelos}{epoch}/doc/')

      model_query.eval()
      model_doc.eval()

      print(f'Loss de treinamento {epoch}: {compute_loss_dataloaders(model_query, dataloader_queries_train, model_doc, dataloader_docs_train)}')
      print(f'Loss de validação {epoch}: {compute_loss_dataloaders(model_query, dataloader_queries_val, model_doc, dataloader_docs_val)}')

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Pesquisa completa no TREC-COVID

Treinado o modelo, agora vamos aplicá-lo ao TREC-COVID:.

Coloca os modelos em modo eval:


In [ ]:
model_query.eval()
model_doc.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

Baixa o trec-covid:

In [ ]:
if not Path('./collections/trec-covid.zip').is_file():
  !wget {url_trec_covid} -P collections # type: ignore
  !unzip -o collections/trec-covid.zip -d ./collections # type: ignore

# Converte o qrels que veio no trec-covid.zip pra o formato esperado:
with open('./collections/trec-covid/qrels/test.tsv', 'r') as fin:
  data = fin.read().splitlines(True)
with open('./collections/trec-covid/qrels/test_corrigido.tsv', 'w') as fout:
  for linha in data[1:]:
    campos = linha.split()
    fout.write(f'{campos[0]}\t0\t{campos[1]}\t{campos[2]}\n')

Carrega as queries e os documentos:

In [ ]:
%%time
import json

def carrega_corpus_trec_covid():
  retorno = []
  with open('./collections/trec-covid/corpus.jsonl') as corpus:
    for i, line in enumerate(corpus):
      doc = json.loads(line)
      #retorno.append({
      #    'id': doc['_id'],
      #    'doc': f"{doc['title']} {doc['text']}"
      #})
      retorno.append(
          (doc['_id'], f"{doc['title']} {doc['text']}")
      )
      if (i % 10000 == 0):
        print(f'Processado {i} documentos')
    return retorno

def carrega_queries_trec_covid():
  retorno = []
  with open('./collections/trec-covid/queries.jsonl') as queries:
    for line in queries:
      query = json.loads(line)
      # Faz apenas uma pequena tradução de _id para id e text para texto
      retorno.append({'id': query['_id'], 'texto': query['text']})
  return retorno

queries_trec_covid = carrega_queries_trec_covid()
corpus_trec_covid = carrega_corpus_trec_covid()

Processado 0 documentos
Processado 10000 documentos
Processado 20000 documentos
Processado 30000 documentos
Processado 40000 documentos
Processado 50000 documentos
Processado 60000 documentos
Processado 70000 documentos
Processado 80000 documentos
Processado 90000 documentos
Processado 100000 documentos
Processado 110000 documentos
Processado 120000 documentos
Processado 130000 documentos
Processado 140000 documentos
Processado 150000 documentos
Processado 160000 documentos
Processado 170000 documentos
CPU times: user 1.83 s, sys: 102 ms, total: 1.93 s
Wall time: 1.9 s


A variável corpus_trec_covid contém os ids e os textos. Agora é necessário carregar a representação vetorial desses textos. Isso será feito gerando a matriz matriz_docs_trec_covid:

In [ ]:
%%time
ids_trec_covid, textos_trec_covid = zip(*corpus_trec_covid)

matriz_docs_trec_covid = None
#textos_trec_covid = textos_trec_covid[0:20000]
dataset_docs_trec_covid = Dataset(tokenizer, textos_trec_covid)
dataloader_docs_trec_covid = DataLoader(dataset_docs_trec_covid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

if gerar_e_salvar_matriz_docs_trec_covid:
  with torch.no_grad():
    for batch in tqdm(dataloader_docs_trec_covid, mininterval=0.5, desc='Convertendo documentos trec-covid', disable=False):
      outputs_docs = model_doc(**batch.to(device))
      tcls_docs    = outputs_docs.last_hidden_state[:, 0, :]
      #tcls_docs    = tcls_docs / torch.norm(tcls_docs, dim=1, keepdim=True)
      # Monta a matriz de documentos na CPU
      tcls_docs = tcls_docs.to("cpu")

      if matriz_docs_trec_covid is None:
        matriz_docs_trec_covid = tcls_docs
      else:
        matriz_docs_trec_covid = torch.cat( (matriz_docs_trec_covid, tcls_docs), dim=0)

  # Agora volta a matriz pra GPU pq ela cabe lá, não sei pq estava estourando a memória antes...
  matriz_docs_trec_covid = matriz_docs_trec_covid.to(device)
  torch.save(matriz_docs_trec_covid, arquivo_matriz_docs_trec_covid)
else:
  matriz_docs_trec_covid = torch.load(arquivo_matriz_docs_trec_covid).to(device)

print(matriz_docs_trec_covid.size())

torch.Size([171332, 384])
CPU times: user 215 ms, sys: 333 ms, total: 549 ms
Wall time: 459 ms


Agora vamos definir um método pra calcular a representação vetorial da query e para fazer a pesquisa na base de dados. O método de pesquisa só retorna o vetor de score pareado com as ids contidas na variável ids_trec_covid:


In [ ]:
def get_vetor_query(query):
  query_tokenizada = tokenizer(query, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

  with torch.no_grad():
    output_query = model_query(**query_tokenizada.to(device))
    tcls_query    = output_query.last_hidden_state[:, 0, :]
    # tcls_query    = tcls_query / torch.norm(tcls_query, dim=1, keepdim=True)

  return tcls_query[0]

vetor_query = get_vetor_query('what is this?')

print(vetor_query.size())


torch.Size([384])


In [ ]:
def calcula_score_documentos_para_a_query(matriz_docs, query):
  vetor_query = get_vetor_query(query)
  score = torch.matmul(matriz_docs, vetor_query)

  return score

def pesquisa_query_e_retorna_n_primeiros_docs(matriz_docs, ids_docs_na_matriz, query, n=1000):
  # Calcula o score
  score = calcula_score_documentos_para_a_query(matriz_docs, query)
  # Ordena
  sorted_score, indices_score = torch.sort(score, descending=True)
  # Pega só os n primeiros
  sorted_score = sorted_score[0:n]
  indices_score = indices_score[0:n]
  # Extrai os ids dos documentos
  ids_docs = [ids_docs_na_matriz[i] for i in indices_score]

  return zip(ids_docs, sorted_score)


Agora roda todas as queries para avaliação...

In [ ]:
# Roda todas as queries
def run_all_queries(file):
  print('Carregando as queries do arquivo queries.jsonl...\n')
  queries_trec_covid = carrega_queries_trec_covid()

  print(f'Total de queries que serão avaliadas: {len(queries_trec_covid)}')
  cnt = 0
  with open(file, 'w') as runfile:
    for query in queries_trec_covid:
      id_query = query['id']
      texto = query['texto']

      if cnt % 5 == 0:
        print(f'{cnt} queries completadas')

      # Pega os primeiros 1000 resultados
      docs_score = pesquisa_query_e_retorna_n_primeiros_docs(matriz_docs_trec_covid, ids_trec_covid, texto, n=1000)

      i = 0
      for id_doc, score in docs_score:
        _ = runfile.write('{} Q0 {} {} {:.6f} Pesquisa_densa\n'.format(id_query, id_doc, i+1, float(score)))
        i += 1

      cnt += 1
      # break # Pra testar, gera só a primeira query
    print(f'{cnt} queries completadas')


E calcula o nDCG@10:

In [ ]:
run_all_queries('run-pesquisa-densa.txt')

!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-pesquisa-densa.txt #type: ignore

Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
5 queries completadas
10 queries completadas
15 queries completadas
20 queries completadas
25 queries completadas
30 queries completadas
35 queries completadas
40 queries completadas
45 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-pesquisa-densa.txt']
Results:
ndcg_cut_10           	all	0.3322


## Pesquisa aproximada (usando k-means)

Vamos pegar a matriz de documentos e criar 10 clusters para eles. E treinar um kmeans:

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Primeiro vamos transformar a matriz de documentos para uma matriz numpy:
matriz_docs_np = matriz_docs_trec_covid.to('cpu').numpy()
matriz_docs_trec_covid.to(device)

# Cálculo dos centróides:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=300, n_init=10, random_state=42)
classe_doc = kmeans.fit_predict(matriz_docs_np)


Agora vamos separar os índices dos documentos nesses 10 clusters e deixar em um dict só pra facilitar as coisas depois:

In [ ]:
indices_por_cluster = {}

for classe in range(0, n_clusters):
  indices_por_cluster[classe] = {
      'indice': [indice for indice, classe_doc in enumerate(classe_doc) if classe == classe_doc],
      'ids_doc': [ids_trec_covid[indice] for indice, classe_doc in enumerate(classe_doc) if classe == classe_doc]
  }
  print(f'Total docs na classe {classe} = {len(indices_por_cluster[classe]["indice"])}')

print(indices_por_cluster[0]['indice'][0:3])
print(indices_por_cluster[0]['ids_doc'][0:3])

Total docs na classe 0 = 28914
Total docs na classe 1 = 12221
Total docs na classe 2 = 24366
Total docs na classe 3 = 22542
Total docs na classe 4 = 11836
Total docs na classe 5 = 11859
Total docs na classe 6 = 22447
Total docs na classe 7 = 8259
Total docs na classe 8 = 15198
Total docs na classe 9 = 13690
[2, 4, 5]
['ejv2xln0', '9785vg6d', 'zjufx4fo']


Vamos definir um método pra, dada uma query (um texto), retornar qual é a classe de documentos que está mais próxima dessa query. Ou seja, esse método extrai o vetor que representa a query e calcula a distância entre esse vetor e o vetor do centro de cada cluster:

In [ ]:
def get_classe_mais_proxima(query):
  vetor_query = get_vetor_query(query).to('cpu').numpy()

  classe_mais_proxima = -1
  dist_mais_proxima = 1e10
  for classe, vetor_classe in enumerate(kmeans.cluster_centers_):
    dist = np.linalg.norm(vetor_query - vetor_classe)
    if dist < dist_mais_proxima:
      dist_mais_proxima = dist
      classe_mais_proxima = classe

  return classe_mais_proxima

get_classe_mais_proxima('What is this?')


7

Agora um método que faz a pesquisa apenas nos documentos que estão no cluster mais próximo.

Como já separamos os ids dos documentos por cluster dentro de indices_por_cluster[classe_mais_proxima]['ids_doc'], o que precisamos agora é só filtrar a matriz matriz_docs_trec_covid pra considerar apenas os documentos equivalentes a indices_por_cluster[classe_mais_proxima]['ids_doc']. Feito isso, basta passar pro método de pesquisa geral:

In [ ]:
def pesquisa_query_na_classe_mais_proxima_e_retorna_n_primeiros_docs(query, n=1000):
  classe_mais_proxima = get_classe_mais_proxima(query)
  indices_na_classe = indices_por_cluster[classe_mais_proxima]['indice']

  matriz_filtrada = matriz_docs_trec_covid[torch.LongTensor(indices_na_classe).to(torch.int)]
  ids_docs_filtrados = indices_por_cluster[classe_mais_proxima]['ids_doc']

  return pesquisa_query_e_retorna_n_primeiros_docs(matriz_filtrada, ids_docs_filtrados, query, n)


In [ ]:
%%time
for i in pesquisa_query_na_classe_mais_proxima_e_retorna_n_primeiros_docs('teste', 10):
  print(i)

('v6nnnzhm', tensor(61.4928, device='cuda:0'))
('crtvxdn6', tensor(58.7026, device='cuda:0'))
('iryg7xw0', tensor(57.4697, device='cuda:0'))
('kyae3q05', tensor(57.1972, device='cuda:0'))
('1qhbucb0', tensor(57.0405, device='cuda:0'))
('zlx4etkg', tensor(55.8098, device='cuda:0'))
('1x3da6ko', tensor(55.6850, device='cuda:0'))
('4oabmb1a', tensor(55.6653, device='cuda:0'))
('4cpiemq5', tensor(55.3849, device='cuda:0'))
('g6sfaa2m', tensor(55.3795, device='cuda:0'))
CPU times: user 43.3 ms, sys: 3.05 ms, total: 46.4 ms
Wall time: 44.6 ms


Como o run_all_queries tinha sido definido pesquisando apenas em todos os documentos, vamos replicar ele aqui mas agora mandando pesquisar apenas na classe mais próxima:

In [ ]:
# Roda todas as queries
def run_all_queries_classe_proxima(file):
  print('Carregando as queries do arquivo queries.jsonl...\n')
  queries_trec_covid = carrega_queries_trec_covid()

  print(f'Total de queries que serão avaliadas: {len(queries_trec_covid)}')
  cnt = 0
  with open(file, 'w') as runfile:
    for query in queries_trec_covid:
      id_query = query['id']
      texto = query['texto']

      if cnt % 5 == 0:
        print(f'{cnt} queries completadas')

      # Pega os primeiros 1000 resultados
      docs_score = pesquisa_query_na_classe_mais_proxima_e_retorna_n_primeiros_docs(texto, n=1000)

      i = 0
      for id_doc, score in docs_score:
        _ = runfile.write('{} Q0 {} {} {:.6f} Pesquisa_densa\n'.format(id_query, id_doc, i+1, float(score)))
        i += 1

      cnt += 1
      # break # Pra testar, gera só a primeira query
    print(f'{cnt} queries completadas')


Por fim, roda a pesquisa e checa o nDCG@10:

In [ ]:
run_all_queries_classe_proxima('run-pesquisa-densa-classe-proxima.txt')

!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-pesquisa-densa-classe-proxima.txt #type: ignore

Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
5 queries completadas
10 queries completadas
15 queries completadas
20 queries completadas
25 queries completadas
30 queries completadas
35 queries completadas
40 queries completadas
45 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-pesquisa-densa-classe-proxima.txt']
Results:
ndcg_cut_10           	all	0.2991


## Usando sentence transformers

In [ ]:
%%time
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-mpnet-base-v2').to(device)

if gerar_arquivo_doc_embeddings_all_mpnet_base_v2:
  docs_embeddings = model.encode(textos_trec_covid)
  torch.save(docs_embeddings, arq_doc_embeddings_all_mpnet_base_v2)
else:
  docs_embeddings = torch.load(arq_doc_embeddings_all_mpnet_base_v2)

CPU times: user 5.13 s, sys: 2.45 s, total: 7.58 s
Wall time: 7.59 s


In [ ]:
def calcula_score_documentos_para_a_query_sentence_transformers(model, docs_embeddings, query):
  query_embedding = model.encode(query)
  return util.dot_score(query_embedding, docs_embeddings)

def pesquisa_query_e_retorna_n_primeiros_docs_sentence_transformers(model, docs_embeddings, query, n=1000):
  # Calcula o score
  score = calcula_score_documentos_para_a_query_sentence_transformers(model, docs_embeddings, query)
  # Ordena
  sorted_score, indices_score = torch.sort(score, descending=True)
  # Pega só os n primeiros
  sorted_score = sorted_score[0, 0:n]
  indices_score = indices_score[0, 0:n]
  # Extrai os ids dos documentos
  ids_docs = [ids_trec_covid[i] for i in indices_score]

  return zip(ids_docs, sorted_score)

# Roda todas as queries
def run_all_queries_sentence_transformer(model, docs_embeddings, file):
  print('Carregando as queries do arquivo queries.jsonl...\n')
  queries_trec_covid = carrega_queries_trec_covid()

  print(f'Total de queries que serão avaliadas: {len(queries_trec_covid)}')
  cnt = 0
  with open(file, 'w') as runfile:
    for query in queries_trec_covid:
      id_query = query['id']
      texto = query['texto']

      if cnt % 5 == 0:
        print(f'{cnt} queries completadas')

      # Pega os primeiros 1000 resultados
      docs_score = pesquisa_query_e_retorna_n_primeiros_docs_sentence_transformers(model, docs_embeddings, texto, n=1000)

      i = 0
      for id_doc, score in docs_score:
        _ = runfile.write('{} Q0 {} {} {:.6f} Pesquisa_densa\n'.format(id_query, id_doc, i+1, float(score)))
        i += 1

      cnt += 1
      # break # Pra testar, gera só a primeira query
    print(f'{cnt} queries completadas')

run_all_queries_sentence_transformer(model, docs_embeddings, 'run-pesquisa-densa-sentence-transformer-all-mpnet-base-v2.txt')
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-pesquisa-densa-sentence-transformer-all-mpnet-base-v2.txt #type: ignore

Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
5 queries completadas
10 queries completadas
15 queries completadas
20 queries completadas
25 queries completadas
30 queries completadas
35 queries completadas
40 queries completadas
45 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-pesquisa-densa-sentence-transformer-all-mpnet-base-v2.txt']
Results:
ndcg_cut_10           	all	0.5133


In [ ]:
%%time
model = SentenceTransformer('all-MiniLM-L12-v2').to(device)

if gerar_arquivo_doc_embeddings_all_MiniLM_L12_v2:
  docs_embeddings = model.encode(textos_trec_covid)
  torch.save(docs_embeddings, arq_doc_embeddings_all_MiniLM_L12_v2)
else:
  docs_embeddings = torch.load(arq_doc_embeddings_all_MiniLM_L12_v2)




CPU times: user 2.38 s, sys: 1.05 s, total: 3.43 s
Wall time: 3.4 s


In [ ]:
%%time
run_all_queries_sentence_transformer(model, docs_embeddings, 'run-pesquisa-densa-sentence-transformer-all-MiniLM-L12-v2.txt')
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-pesquisa-densa-sentence-transformer-all-MiniLM-L12-v2.txt #type: ignore

Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
5 queries completadas
10 queries completadas
15 queries completadas
20 queries completadas
25 queries completadas
30 queries completadas
35 queries completadas
40 queries completadas
45 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-pesquisa-densa-sentence-transformer-all-MiniLM-L12-v2.txt']
Results:
ndcg_cut_10           	all	0.5082
CPU times: user 7.42 s, sys: 7.18 s, total: 14.6 s
Wall time: 13.4 s
